In [ ]:
function solve_MIP(kep_graph)
    #modele = Model(Gurobi.Optimizer)
    modele = Model(GLPK.Optimizer)
    
    # Nombre de paires de donneurs/patients, nombre de noeuds du graphe :
    P=size(new_inst.pairs)[1]
    # Longueur maximale d'un cycle
    L=new_inst.max_cycle_length
    
    # Variable : indique si le patient de la paire j a un rein du donneur de la paire i
    @variable(modele,x[i in 1:P,j in 1:P],Bin)
    # Variable : indique si l'arc (i,j) appartient au graphe ou non
    @variable(modele,w[i in 1:P,j in 1:P],Bin)
    
    # Contrainte : le patient i recoit un rein ssi le donneur i donne un rein
    @constraint(modele,recoit[i in 1:P],sum(x[j,i] for j in 1:P if (new_inst.edge_weight[j,i]==1.0))==sum(x[i,j] for j in 1:P if (new_inst.edge_weight[i,j]==1.0)))
    # Contrainte : un donneur peut donner seulement un rein
    @constraint(modele,capacite[i in 1:P],sum(x[i,j] for j in 1:P if (new_inst.edge_weight[i,j]==1.0))<=1)
    # Contrainte : on veut des cycles plus petits que L
    
    # Objectif : maximiser le nombre total de transplantations
    @objective(modele,Max,sum(w[i,j]*x[i,j] for i in 1:P,j in 1:P if (new_inst.edge_weight[i,j]==1.0)))
    
    set_optimizer_attribute(modele, MOI.Silent(), true)
    optimize!(modele)

    return JuMP.objective_value(modele),JuMP.value.(x)
end